In [1]:
import io
import re
import pandas as pd

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from PyPDF2 import PdfFileReader, PdfFileWriter

report_path = r'example_punch.pdf'
sorted_report_path = r'sorted_report.pdf'
mapping_path = r'mapping.csv'

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [2]:
#extract the text from the pdf
report_txt = convert_pdf_to_txt(report_path)

In [3]:
#remove the cids
cid_pattern = re.compile(r'\(cid:\d\d\)\n')
cid_removed = re.sub(cid_pattern, '', report_txt)

#clean up the emp number
emp_id_pattern = re.compile(r'Emp\s#\s')
emp_id_fix = r'EmpID '
emp_fixed = re.sub(emp_id_pattern, emp_id_fix, cid_removed)

In [4]:
#replace all newlines w/ space
cleaned = emp_fixed.replace('\n', ' ')


In [5]:
#get all the employee ids
ids = re.findall(r'EmpID\s[\d]+', cleaned)

In [50]:
#put emp ids in a df
emp_id_df = pd.DataFrame({'Emp_ID': ids})

In [51]:
#extract the numbers
emp_id_df['Emp_ID'] = emp_id_df['Emp_ID'].str.extract(r'(\d+)').astype(int)

In [52]:
emp_id_df

,Emp_ID
0,60
1,765
2,813
3,63
4,760
5,857
6,50
7,45
8,770
9,860


In [53]:
#get the number of pages of employee timesheets
emp_pages = len(emp_id_df)
emp_pages

30

In [54]:
#get the total number of pages
with open(report_path, 'rb') as infile:
    reader = PdfFileReader(infile)
    total_pages = reader.getNumPages()
total_pages    

34

In [55]:
#calculate the report page length
report_pages = total_pages - emp_pages
report_pages

4

In [56]:
#add the report page(s)
rep_length = report_pages
while rep_length:
    place = 0 - rep_length
    emp_id_df = emp_id_df.append({'Emp_ID': place}, ignore_index=True)
    rep_length -= 1
emp_id_df

,Emp_ID
0,60
1,765
2,813
3,63
4,760
5,857
6,50
7,45
8,770
9,860


In [57]:
emp_id_df.shape

(34, 1)

In [58]:
#read mapping csv to a df
mapping_df = pd.read_csv(mapping_path)

In [59]:
print(mapping_df.shape)
mapping_df.head()

(270, 2)


,DC Clock Number,MAS Number
0,224,14-0000046
1,225,14-0000440
2,105,16-0000001
3,103,16-0000004
4,733,16-0000093


In [60]:
#rename the columns
mapping_df.columns = ['Emp_ID', 'Sort_Key']
mapping_df.head()

,Emp_ID,Sort_Key
0,224,14-0000046
1,225,14-0000440
2,105,16-0000001
3,103,16-0000004
4,733,16-0000093


In [61]:
#left outer join the two df 
combo_df = pd.merge(emp_id_df, mapping_df, on='Emp_ID', how='left')
print(combo_df.shape)
combo_df

(34, 2)


,Emp_ID,Sort_Key
0,60,NaN
1,765,17-0000858
2,813,NaN
3,63,NaN
4,760,NaN
5,857,NaN
6,50,NaN
7,45,NaN
8,770,NaN
9,860,NaN


In [62]:
#change all NaN to large number string
combo_df = combo_df.fillna('9999-999999999')
combo_df

,Emp_ID,Sort_Key
0,60,9999-999999999
1,765,17-0000858
2,813,9999-999999999
3,63,9999-999999999
4,760,9999-999999999
5,857,9999-999999999
6,50,9999-999999999
7,45,9999-999999999
8,770,9999-999999999
9,860,9999-999999999


In [63]:
#remove the dash in Sort_Key
combo_df['Sort_Key'] = combo_df['Sort_Key'].str.replace('-', '', regex=False)
combo_df

,Emp_ID,Sort_Key
0,60,9999999999999
1,765,170000858
2,813,9999999999999
3,63,9999999999999
4,760,9999999999999
5,857,9999999999999
6,50,9999999999999
7,45,9999999999999
8,770,9999999999999
9,860,9999999999999


In [64]:
#convert Sort_Key to numbers
combo_df['Sort_Key'] = pd.to_numeric(combo_df['Sort_Key'], errors='coerce')
combo_df

,Emp_ID,Sort_Key
0,60,9999999999999
1,765,170000858
2,813,9999999999999
3,63,9999999999999
4,760,9999999999999
5,857,9999999999999
6,50,9999999999999
7,45,9999999999999
8,770,9999999999999
9,860,9999999999999


In [65]:
combo_df['Sort_Key'].describe()

count    3.400000e+01
mean     7.353029e+12
std      4.477962e+12
min      1.700008e+08
25%      2.500413e+12
50%      1.000000e+13
75%      1.000000e+13
max      1.000000e+13
Name: Sort_Key, dtype: float64

In [66]:
#change it so the report will come first in the df
while report_pages:
    place = 0 - report_pages
    combo_df.loc[combo_df.Emp_ID == place, 'Sort_Key'] = place
    report_pages -= 1
combo_df

,Emp_ID,Sort_Key
0,60,9999999999999
1,765,170000858
2,813,9999999999999
3,63,9999999999999
4,760,9999999999999
5,857,9999999999999
6,50,9999999999999
7,45,9999999999999
8,770,9999999999999
9,860,9999999999999


In [67]:
#sort by Sort_Key then Emp_ID
combo_df.sort_values(by=['Sort_Key', 'Emp_ID'], inplace=True)

In [68]:
combo_df

,Emp_ID,Sort_Key
30,-4,-4
31,-3,-3
32,-2,-2
33,-1,-1
13,761,170000775
1,765,170000858
12,850,170001110
19,205,250000375
21,214,270000087
25,210,310000054


In [69]:
#put the page order into a list
page_order = combo_df.index.tolist()

In [70]:
print(page_order)

[30, 31, 32, 33, 13, 1, 12, 19, 21, 25, 24, 20, 23, 10, 7, 6, 11, 0, 3, 22, 18, 29, 26, 27, 28, 17, 4, 15, 8, 2, 5, 9, 16, 14]


In [71]:
#reorder the pages to a new pdf
writer = PdfFileWriter()
with open(report_path, 'rb') as infile:
    
    reader = PdfFileReader(infile)
    for entry in page_order:
        writer.addPage(reader.getPage(entry))

    with open(sorted_report_path, 'wb') as outfile:
        writer.write(outfile)